In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities = pd.read_html(r"C:\Users\olona\OneDrive\Desktop\abdul\assignments\assignment4\assests\wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities.rename(columns={"Population (2016 est.)[8]": "Population"},inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")



def nhl_df():
    Big4='NHL'
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area'] #pass the metropolitan column
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
    team=pd.merge(team,cities,how='left',on='Metropolitan area').iloc[:,1:4]
    team=team.drop(index=28)
     #drop col 28 bcos it is not needed and will affect our calculation
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int}) #declare the dtypes
    team['team']=team['team'].str.replace('[\w.]*\ ','') #replace first words in the list withis format
    _df=pd.read_csv(r"C:\Users\olona\OneDrive\Desktop\abdul\assignments\assignment4\assests\nhl.csv")
    _df = _df[_df['year'] == 2018]
        #as specified, we are only usig rows from the year 2018
    _df['team'] = _df['team'].str.replace(r'\*',"")#replace * with whitespace
    _df['team'] = _df['team'].str.replace(r'[\xa0]',"")#remove non breaking spaces so it does not process internally as \xa0 and affect calc
    _df = _df[['team','W','L']] #pick out the useful cols
    dropList=[]
    for i in range(_df.shape[0]):
        row=_df.iloc[i]
        if row['team']==row['W'] and row['L']==row['W']:
                print(row['team'],'will be dropped!')
                dropList.append(i)
    _df=_df.drop(dropList)
    _df['team'] = _df['team'].str.replace('[\w.]* ','')
    _df = _df.astype({'team': str,'W': int, 'L': int}) #declare dtype to stay on course
    _df['W/L%'] = _df['W']/(_df['W']+_df['L']) #calculation of win loss ratio
    merge=pd.merge(team,_df,'outer', on = 'team')
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    return merge[['W/L%']]

def nba_df():
    Big4='NBA'
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team= pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace('',np.nan).replace('-',np.nan).dropna().reset_index().rename(columns={'value':'teams'})
    team=pd.merge(team,cities,how='left',on='Metropolitan area').loc[:,['Metropolitan area','teams','Population']]
    team=team.drop(index=28)
    team = team.astype({'Metropolitan area': str, 'teams': str, 'Population': int})
    team['teams']=team['teams'].str.replace('[\w.]*\ ','')
    _df=pd.read_csv(r"C:\Users\olona\OneDrive\Desktop\abdul\assignments\assignment4\assests\nba.csv")
    _df = _df[_df['year'] == 2018]
    _df=_df.rename(columns={'team':'teams'})
    _df['teams'] = _df['teams'].str.replace(r'[\*]',"")
    _df['teams'] = _df['teams'].str.replace(r'\(\d*\)',"")
    _df['teams'] = _df['teams'].str.replace(r'[\xa0]',"") #remove non breaking spaces
    _df=_df[['teams','W/L%']]
    _df['teams']=_df['teams'].str.replace('[\w]* ','')
    _df = _df.astype({'teams': str,'W/L%': float}) #always declare dtype after selecting cols
    merge=pd.merge(team,_df,how='outer',on='teams')
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    return merge[['W/L%']]

def mlb_df():
    Big4='MLB'
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']= cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
    team=pd.merge(team,cities,how='left',on = 'Metropolitan area').iloc[:,1:4]
    team=team.drop(index=26)   
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('\ Sox','Sox')
    _df=pd.read_csv(r"C:\Users\olona\OneDrive\Desktop\abdul\assignments\assignment4\assests\mlb.csv")
    _df = _df[_df['year'] == 2018]
    _df['team'] = _df['team'].str.replace(r'[\*]',"")
    _df['team'] = _df['team'].str.replace(r'\(\d*\)',"")
    _df['team'] = _df['team'].str.replace(r'[\xa0]',"")
    _df = _df[['team','W-L%']]
    _df.rename(columns={"W-L%": "W/L%"},inplace=True)
    _df['team']=_df['team'].str.replace('\ Sox','Sox')
    _df['team'] = _df['team'].str.replace('[\w.]* ','')
    _df = _df.astype({'team': str,'W/L%': float})
    _df

    merge=pd.merge(team,_df,'outer', on = 'team')
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

    merge=merge.replace(np.nan,'0.451')
    return merge[['W/L%']]

def nfl_df():
    Big4='NFL'
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team=pd.melt(team, id_vars=['Metropolitan area']).drop(columns='variable').replace('',np.nan).replace('-','').dropna().reset_index().rename(columns={'value':'team'})
    team=team.drop(index=29)
    team=pd.merge(team,cities,how='left',on='Metropolitan area').iloc[:,1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    _df=pd.read_csv(r"C:\Users\olona\OneDrive\Desktop\abdul\assignments\assignment4\assests\nfl.csv")
    _df=_df[_df['year']==2018]
    _df=_df[['team','W-L%']]
    _df=_df.rename(columns={'W-L%':'W/L%'})

    droplist=[]
    for i in range(_df.shape[0]):
        row=_df.iloc[i]
        if row['team']==row['W/L%']:
            droplist.append(i)
    _df=_df.drop(droplist)
    _df['team'] = _df['team'].str.replace('[\w.]* ','')
    _df['team'] = _df['team'].str.replace('+','')
    _df['team'] = _df['team'].str.replace(r'[\xa0]',"")
    _df['team'] = _df['team'].str.replace(r'\*',"")
    _df=_df.astype({'team': str,'W/L%': float})
    merge=pd.merge(_df,team,how='outer',on='team')
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    return merge[['W/L%']]
    

def create_df(sport):
    if sport =='NFL':
        return nfl_df()
    elif sport =='NBA':
        return nba_df()
    elif sport =='NHL':
        return nhl_df()
    elif sport =='MLB':
        return mlb_df()
    else:
        print("ERROR with intput!")


        # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
        # df.loc["NFL","NFL"] should return np.nan
sports = ['NFL', 'NBA', 'NHL', 'MLB']
p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
        
for i in sports:
    for j in sports:
        if i is not j :
            merge=pd.merge(create_df(i),create_df(j),'inner', on = ['Metropolitan area'])
            p_values=stats.ttest_rel(merge['W/L%_x'],merge['W/L%_y'])[0]
                      
p_values
                

